# IBM: Applied Data Science Capstone

# Assignment: Segmenting and Clustering Neighborhoods in Toronto

# Author: Paschoal

# Date: 2021-02-16


_______________________________________________________________________________________________________

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#jump1">Installing and Importing libraries</a>

2.  <a href="#jump2">Creating the Pandas dataframe</a>

3.  <a href="#jump3">Building DataFrame with Geospatial Data</a>

4.  <a href="#jump4">Explore and cluster the neighborhoods in Toronto</a>
    </font>
    </div>


<a id="jump1"></a>

# 1. Installing and Importing libraries

In [1]:
# importing libraries
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import urllib.request, urllib.parse, urllib.error
import ssl

# Installing beautifulsoup4
# !pip install beautifulsoup4 # uncomment this line if beautifulsoup4 is not already installed

from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

# !pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !pip install folium
import folium # map rendering library

print('\n Libraries imported.')


 Libraries imported.


<a id="jump2"></a>

# 2. Creating the Pandas dataframe

In [2]:
# Reading and parsing the Wikipedia page using Beatifulsoup

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# Read and parse the wikipedia webpage
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')

In [3]:
# getting the wikipedia table
table = soup.find("table", { "class" : "wikitable sortable" })

# creating a list of columns names
columns_names = ["Postal Code", "Borough", "Neighborhood"]
    
# create an empty list Table of Postal Codes (Table_PC) 
Table_PC = []

# finding and preparing rows, and append to the Table_PC
for row in table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) != 0:
        if cells[1].find(text=True) !='Not assigned\n':   # eliminating 'Not assigned' Borough rows
            table_row = []
            for i in range(0,3):
                tr_iten = cells[i].find(text=True)
                table_row.append(tr_iten[0:-1])
            Table_PC.append(table_row)

In [4]:
# Creating a pandas dataframe 
df = pd.DataFrame(Table_PC)
df.columns = columns_names 
print(df['Postal Code'].is_unique)
df.describe()

True


,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M4J,North York,Downsview
freq,1,24,4


In [5]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
# Shape of the DataFrame
print('Shape', df.shape)

Shape (103, 3)


<a id="jump3"></a>

# 3. Building DataFrame with Geospatial Data

In [7]:
# reading csv file to temporary dataframe
df2 = pd.read_csv("Geospatial_Coordinates.csv")
print(df2.shape)
df2.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
# building a postal code dictionary for fast search (hash table)
ll_dict = dict()

for index, row in df2.iterrows():
    ll_dict[row[0]] = (row[1],row[2]) 

In [9]:
# building two lists to be added to the dataframe
# create both as empty lists

lati_list = []  # Latitude list
longi_list = []  # Longitude list

# loop until get all geospatial coordinates from dictionay

for index, row in df.iterrows():
    lati_list.append(ll_dict[row[0]][0])   # append Latitude information from dictionary 
    longi_list.append(ll_dict[row[0]][1])   # append Longitude information from dictionary
            
# adding two columns (Latitude and Longitude lists) to the dataframe 

df['Latitude'] = lati_list
df['Longitude'] = longi_list

# New dataframe with geospatial location
print('Shape',df.shape)
df.head()

Shape (103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<a id='jump4'></a>

# 4. Explore and cluster the neighborhoods in Toronto

In [10]:
# droping unnecessary information (Postal Code)
df.drop("Postal Code", axis=1, inplace=True)

# creating the basic neighborhoods dataframe
toronto_data = df

#verifying the shape and content
print(toronto_data.shape)
toronto_data.head()

(103, 4)


,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [11]:
# finding the Toronto geospatial location
address = 'City of Toronto'

geolocator = Nominatim(user_agent="tot_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


## Lets visualize the Neighborhoods in Toronto

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Defining Foursquare credentials and API version

In [198]:
# Defining Foursquare credentials and API version

CLIENT_ID = 'YOUR_CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'YOUR_CLIENT_SECRET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YOUR_CLIENT_ID
CLIENT_SECRET:YOUR_CLIENT_SECRET


## Let's create a function to get all venues from all the neighborhoods

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, end=' | ')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(toronto_data['Neighborhood'], toronto_data['Latitude'],  toronto_data['Longitude'], radius=500)

Parkwoods | Victoria Village | Regent Park, Harbourfront | Lawrence Manor, Lawrence Heights | Queen's Park, Ontario Provincial Government | Islington Avenue, Humber Valley Village | Malvern, Rouge | Don Mills | Parkview Hill, Woodbine Gardens | Garden District, Ryerson | Glencairn | West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale | Rouge Hill, Port Union, Highland Creek | Don Mills | Woodbine Heights | St. James Town | Humewood-Cedarvale | Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood | Guildwood, Morningside, West Hill | The Beaches | Berczy Park | Caledonia-Fairbanks | Woburn | Leaside | Central Bay Street | Christie | Cedarbrae | Hillcrest Village | Bathurst Manor, Wilson Heights, Downsview North | Thorncliffe Park | Richmond, Adelaide, King | Dufferin, Dovercourt Village | Scarborough Village | Fairview, Henry Farm, Oriole | Northwood Park, York University | East Toronto, Broadview North (Old East York) | Harbourfront East, Union Station, Toro

## Let´s see the Toronto Venues dataframe

In [82]:
print('Shape', toronto_venues.shape)
print(len(toronto_venues["Neighborhood"].unique()))
toronto_venues.head()

Shape (2117, 7)
97


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Towns On The Ravine,43.754754,-79.332552,Hotel
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [83]:
# Save Toronto venues data in a csv file
toronto_venues.to_csv("toronto_venues.csv")

## Let´s exclude small neighborhood with less than 10 venues

In [143]:
# Creating a dictionary for fast search of venues count per neighborhood
venues_count = toronto_venues.groupby('Neighborhood').count()
venues_count = dict(venues_count['Venue'])

# search and drop rows with small neighborhoods (<11 venues)
for index, row in toronto_venues.iterrows():
    if venues_count[row[0]] < 11:
        toronto_venues.drop(index, inplace=True)

In [179]:
# Visualize Toronto venues data after exclusion
print(len(toronto_venues["Neighborhood"].unique()))
print(toronto_venues.shape)
toronto_venues.head()

47
(1905, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
7,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
8,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
10,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
11,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's find out how many unique categories can be curated from all the returned venues

In [180]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} uniques venue brands.'.format(len(toronto_venues['Venue'].unique())))

There are 250 uniques categories.
There are 1224 uniques venue brands.


## Analyze Each Neighborhood

In [181]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [182]:
toronto_onehot.shape

(1905, 250)

In [183]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print('Shape',toronto_grouped.shape)
toronto_grouped.head()

Shape (47, 250)


,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.095238,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0000,0.047619,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.047619,0.0,0.0,0.000000,0.000000,0.000000,0.095238,0.0,0

##  let's create the new dataframe and display the top 10 venues for each neighborhood.

In [184]:
# function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [185]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print('Dataframe Shape', neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

Dataframe Shape (47, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Sushi Restaurant,Pharmacy,Pizza Place,Mobile Phone Shop,Middle Eastern Restaurant,Restaurant,Bridal Shop,Deli / Bodega
1,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Thai Restaurant,Pizza Place,Juice Bar,Liquor Store,Restaurant,Indian Restaurant,Fast Food Restaurant
2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Cheese Shop,Restaurant,Farmers Market,Jazz Club,Basketball Stadium
3,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Coffee Shop,Breakfast Spot,Yoga Studio,Climbing Gym,Performing Arts Venue,Pet Store,Convenience Store,Burrito Place
4,"Business reply mail Processing Centre, South C...",Park,Recording Studio,Fast Food Restaurant,Farmers Market,Light Rail Station,Auto Workshop,Restaurant,Comic Shop,Spa,Pizza Place


## Cluster Neighborhoods

In [186]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 3, 2, 2, 0, 0, 3, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [187]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

for index, row in toronto_data.iterrows():
    try: 
        if venues_count[row[1]] < 11:
            toronto_data.drop(index, inplace=True)
    except:
        toronto_data.drop(index, inplace=True)
        
toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(10) # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Café,Breakfast Spot,Theater,Brewery,Spa,Cosmetics Shop
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Vietnamese Restaurant,Miscellaneous Shop,Event Space,Boutique,Dessert Shop,Deli / Bodega
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Music Venue,Sandwich Place,Bar,Italian Restaurant,Beer Bar,Smoothie Shop,Japanese Restaurant,College Auditorium
7,North York,Don Mills,43.745906,-79.352188,2,Gym,Coffee Shop,Clothing Store,Japanese Restaurant,Restaurant,Beer Store,Shopping Mall,Sporting Goods Shop,Italian Restaurant,Bike Shop
8,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,2,Pizza Place,Café,Pet Store,Bank,Intersection,Athletics & Sports,Pharmacy,Flea Market,Gym / Fitness Center,Gastropub
9,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Café,Middle Eastern Restaurant,Theater,Electronics Store
13,North York,Don Mills,43.725900,-79.340923,2,Gym,Coffee Shop,Clothing Store,Japanese Restaurant,Restaurant,Beer Store,Shopping Mall,Sporting Goods Shop,Italian Restaurant,Bike Shop
15,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Department Store,Italian Restaurant,Gym,Restaurant,Bakery
20,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Cheese Shop,Restaurant,Farmers Market,Jazz Club,Basketball Stadium
23,East York,Leaside,43.709060,-79.363452,2,Sporting Goods Shop,Coffee Shop,Bank,Furniture / Home Store,Shopping Mall,Burger Joint,Beer Store,Electronics Store,Liquor Store,Sports Bar


In [188]:
# Deleting row with Cluster Labels NaN
toronto_merged.dropna(subset = ['Cluster Labels'], axis=0, inplace=True)
toronto_merged.shape

(51, 15)

Finally, let's visualize the resulting clusters

In [189]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

The first Cluster

In [192]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Pub,Café,Breakfast Spot,Theater,Brewery,Spa,Cosmetics Shop
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Music Venue,Sandwich Place,Bar,Italian Restaurant,Beer Bar,Smoothie Shop,Japanese Restaurant,College Auditorium
15,St. James Town,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Department Store,Italian Restaurant,Gym,Restaurant,Bakery
20,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Cheese Shop,Restaurant,Farmers Market,Jazz Club,Basketball Stadium
24,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Bubble Tea Shop,Burger Joint,Yoga Studio,Comic Shop,Restaurant
30,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Thai Restaurant,Deli / Bodega,Clothing Store,Bakery,Gym,Asian Restaurant,Pizza Place
36,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Fried Chicken Joint,Restaurant,Brewery,Italian Restaurant,Sporting Goods Shop
42,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Seafood Restaurant,Italian Restaurant,Restaurant,American Restaurant,Salad Place,Japanese Restaurant,Steakhouse
48,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Gym,American Restaurant,Seafood Restaurant,Cocktail Bar,Japanese Restaurant
54,Studio District,Coffee Shop,Gastropub,Bakery,Brewery,Café,American Restaurant,Yoga Studio,Bookstore,Pizza Place,Cheese Shop


The first cluster is the downtown area. Lots of food related venues.

The second Cluster

In [193]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,Downsview,Park,Athletics & Sports,Liquor Store,Snack Place,Baseball Field,Bank,Business Service,Food Truck,Hotel,Shopping Mall
46,Downsview,Park,Athletics & Sports,Liquor Store,Snack Place,Baseball Field,Bank,Business Service,Food Truck,Hotel,Shopping Mall
53,Downsview,Park,Athletics & Sports,Liquor Store,Snack Place,Baseball Field,Bank,Business Service,Food Truck,Hotel,Shopping Mall
60,Downsview,Park,Athletics & Sports,Liquor Store,Snack Place,Baseball Field,Bank,Business Service,Food Truck,Hotel,Shopping Mall


The second cluster is the airport area.

The third Cluster

In [194]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Don Mills,Gym,Coffee Shop,Clothing Store,Japanese Restaurant,Restaurant,Beer Store,Shopping Mall,Sporting Goods Shop,Italian Restaurant,Bike Shop
8,"Parkview Hill, Woodbine Gardens",Pizza Place,Café,Pet Store,Bank,Intersection,Athletics & Sports,Pharmacy,Flea Market,Gym / Fitness Center,Gastropub
9,"Garden District, Ryerson",Clothing Store,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Café,Middle Eastern Restaurant,Theater,Electronics Store
13,Don Mills,Gym,Coffee Shop,Clothing Store,Japanese Restaurant,Restaurant,Beer Store,Shopping Mall,Sporting Goods Shop,Italian Restaurant,Bike Shop
23,Leaside,Sporting Goods Shop,Coffee Shop,Bank,Furniture / Home Store,Shopping Mall,Burger Joint,Beer Store,Electronics Store,Liquor Store,Sports Bar
28,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Sushi Restaurant,Pharmacy,Pizza Place,Mobile Phone Shop,Middle Eastern Restaurant,Restaurant,Bridal Shop,Deli / Bodega
29,Thorncliffe Park,Sandwich Place,Indian Restaurant,Yoga Studio,Pharmacy,Coffee Shop,Restaurant,Burger Joint,Intersection,Bus Line,Fast Food Restaurant
33,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Japanese Restaurant,Bank,Bakery,Food Court,Juice Bar,Toy / Game Store
41,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Bubble Tea Shop,Bakery,Pub,Pizza Place
47,"India Bazaar, The Beaches West",Fast Food Restaurant,Fish & Chips Shop,Brewery,Pub,Sushi Restaurant,Restaurant,Italian Restaurant,Steakhouse,Pizza Place,Liquor Store


The third cluster looks the residential area.

The fourth Cluster

In [196]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub
31,"Dufferin, Dovercourt Village",Pharmacy,Bakery,Park,Middle Eastern Restaurant,Bank,Café,Bar,Pool,Supermarket,Brewery
37,"Little Portugal, Trinity",Bar,Asian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Men's Store,Café,Greek Restaurant,Malay Restaurant,Italian Restaurant,Japanese Restaurant
43,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Coffee Shop,Breakfast Spot,Yoga Studio,Climbing Gym,Performing Arts Venue,Pet Store,Convenience Store,Burrito Place
69,"High Park, The Junction South",Mexican Restaurant,Café,Thai Restaurant,Arts & Crafts Store,Park,Fried Chicken Joint,Bookstore,Discount Store,Diner,Italian Restaurant
80,"University of Toronto, Harbord",Café,Bakery,Bookstore,Bar,Japanese Restaurant,Bank,Italian Restaurant,Beer Bar,Beer Store,College Arts Building
84,"Kensington Market, Chinatown, Grange Park",Café,Vegetarian / Vegan Restaurant,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Bar,Bakery,Dessert Shop,Gaming Cafe,Park


The fourth Cluster is a residential or mixed area close to downtown.

The fourth Cluster

In [197]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Vietnamese Restaurant,Miscellaneous Shop,Event Space,Boutique,Dessert Shop,Deli / Bodega


The fourth is an neighborhood close to the airport

### Thanks for your atention!